<a href="https://colab.research.google.com/github/JimMiller-0/Might-be-my-Year/blob/main/Might_be_my_Year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook for retreiving Fantasy Football Data from ESPN for analysis and predictive capabilities

Get Data

In [ ]:
# Install necessary SDKs

!pip install espn-api
!pip install google-cloud-secret-manager





Imports

In [1]:
import pandas as pd
from espn_api.football import League
from google.cloud import secretmanager
import requests
import json
import time
import numpy as np
import datetime
import os

League Variables & Access Tokens for ESPN API

In [3]:
# You will need to get your league ID and ESPN S2 and SWID
# See https://github.com/cwendt94/espn-api/wiki/Football-Intro for details
# Recommended: Store SWID and ESPN S2 in a secrets manager, like gcp secrets manager: https://cloud.google.com/security/products/secret-manager

league_id = 1054374 # => set to league ID that you want to pull data from
season=2023 # => set to year you want to pull data from
url=f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?scoringPeriodId=17&view=mBoxscore&view=m' # => url of ESPN API. Note: this has changed over the years, navigate to the site and inspect network calls to get current endpoint

# Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()

#Create a Client for secrets manager
client = secretmanager.SecretManagerServiceClient()
project_id = 'might-be-my-year' # => GCP Project ID where secrets manager is enabled
secret_espn_s2 = 'espn_s2' # name of secret in GCP secrets manager for espn_s2
secret_swid = 'swid' # name of secret in GCP secrets manager for swid

# Forge the paths to the latest version of the secrets with a F-string:
resource_name_espn_s2 = f"projects/{project_id}/secrets/{secret_espn_s2}/versions/latest"
resource_name_swid = f"projects/{project_id}/secrets/{secret_swid}/versions/latest"

# Load up the secrets to a variable at runtime:
response_espn_s2 = client.access_secret_version(name=resource_name_espn_s2)
response_swid = client.access_secret_version(name=resource_name_swid)

espn_s2 = response_espn_s2.payload.data.decode("UTF-8")
swid = response_swid.payload.data.decode("UTF-8")





Get Data From ESPN API

In [ ]:
r = requests.get(url, cookies={'swid': swid, 'espn_s2': espn_s2})
data = r.json()
data